# Search AiiDA Database Slab optimizations

In [ ]:
%aiida
import ipywidgets as ipw
from IPython.display import clear_output
from apps.surfaces.widgets.search_completed_widget import SearchCompletedWidget
from apps.surfaces.widgets import search_old_completed_widget

In [ ]:
old = ipw.Checkbox(
    value=False,
    description='OLD workchains',
    disabled=False,
    indent=False
)
output = ipw.Output()
def select_version(_=None):
    if old.value:
        PREPROC_VERSION = 1.20
        s=search_old_completed_widget.SearchCompletedWidget(
            version=PREPROC_VERSION,
            wlabel='SlabGeoOptWorkChain',
            clabel='slab_geo_opt',
            fields_disable={}
        )     
    else:
        s = SearchCompletedWidget(
            wlabel='Cp2kSlabOptWorkChain',
            fields_disable={}
        )
    with output:
        clear_output()
        display(ipw.VBox([old,s]))
        
old.observe(select_version, names='value')
display(output)
select_version()